In [1]:
import os
import shutil
import numpy as np
import tensorflow as tf
from PIL import Image
import tensorflow.keras as keras

2022-10-17 11:33:11.573975: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-17 11:33:11.707938: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-17 11:33:12.180943: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2022-10-17 11:33:12.180998: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_

In [41]:
import tensorflow as tf
print(tf.__version__)

# Set CPU as available physical device
my_devices = tf.config.experimental.list_physical_devices(device_type='CPU')
#tf.config.experimental.set_visible_devices(devices= my_devices, device_type='CPU')

#tf.config.set_visible_devices([], 'GPU')
print(" ", len(tf.config.list_physical_devices()))

2.10.0
  2


In [38]:
os.environ["CUDA_VISIBLE_DEVICES"]="-1" 

try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        print(device)
        assert device.device_type != 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [45]:
# set the batch size
batch_size = 2
image_size = (128, 128)

In [46]:
import keras.backend as K

dtype='float16'
K.set_floatx(dtype)

# default is 1e-7 which is too small for float16.  Without adjusting the epsilon, we will get NaN predictions because of divide by zero problems
K.set_epsilon(1e-4)

In [50]:
# Create the base model from the pre-trained model MobileNet V2
image_shape = image_size + (3,)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
#os.environ["CUDA_VISIBLE_DEVICES"]="-1" 
tf.device("cpu:0")
#tf.device("gpu:0")
if False:
     
    base_model = tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=image_shape,
    pooling=None,
    classes=2,
    classifier_activation='softmax',)
if False:
    base_model = tf.keras.applications.resnet50.ResNet50(
            include_top=False,
            weights='imagenet',
            input_tensor=None,
            input_shape=image_shape,
            pooling=None,
            classes=2,
            #**kwargs
        )
if False:
    base_model = tf.keras.applications.MobileNet(input_shape=image_shape,
                                              include_top=False,
                                              weights='imagenet')

Num GPUs Available:  1


In [48]:
flowerdirs = os.listdir("2flowers")
destdir = "128_flowers"
if os.path.isdir(destdir):
    shutil.rmtree(destdir)
if not os.path.isdir(destdir):
    os.mkdir(destdir)
    
subdirs = os.listdir(destdir)
for flowerdir in flowerdirs:
    print(flowerdir)
    subdir = os.path.join(destdir, flowerdir)
    if not os.path.exists(subdir):
        os.makedirs(subdir)

    for i, file in  enumerate(os.listdir(os.path.join("2flowers", flowerdir))):
        fpath = os.path.join("2flowers", flowerdir, file)
        orig = Image.open(fpath)
        temp = orig.convert("RGB")
        small = temp.resize(image_size)
        small.save(os.path.join(destdir, flowerdir, file))


sunflowers
daisy


In [7]:
# create the training data set
train_ds = tf.keras.utils.image_dataset_from_directory(
    "128_flowers",
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    seed=145,
    validation_split=0.2,
    subset="training",
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 1332 files belonging to 2 classes.
Using 1066 files for training.


In [8]:
# double check the class names
class_names = train_ds.class_names
print(class_names)

['daisy', 'sunflowers']


In [9]:
# create the validation data set
val_ds = tf.keras.utils.image_dataset_from_directory(
    "128_flowers",
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    seed=145,
    validation_split=0.2,
    subset="validation",
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 1332 files belonging to 2 classes.
Using 266 files for validation.


In [10]:
if False:
    data_augmentation = tf.keras.Sequential([
      tf.keras.layers.RandomFlip('horizontal'),
      tf.keras.layers.RandomRotation(0.2),
    ])

In [11]:
preprocess_input = tf.keras.applications.mobilenet.preprocess_input
rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)

In [12]:
tf.__version__

'2.10.0'

In [13]:
image_batch, label_batch = next(iter(train_ds))
print(len(image_batch), len(label_batch))

8 8


In [14]:
image_batch

<tf.Tensor: shape=(8, 128, 128, 3), dtype=float32, numpy=
array([[[[ 47.,  71.,  49.],
         [ 44.,  69.,  47.],
         [ 32.,  57.,  35.],
         ...,
         [ 33.,  43.,  32.],
         [ 30.,  40.,  29.],
         [ 28.,  38.,  27.]],

        [[ 41.,  65.,  43.],
         [ 46.,  71.,  49.],
         [ 30.,  58.,  35.],
         ...,
         [ 38.,  44.,  34.],
         [ 40.,  46.,  36.],
         [ 40.,  46.,  36.]],

        [[ 57.,  82.,  60.],
         [ 52.,  80.,  57.],
         [ 22.,  52.,  28.],
         ...,
         [ 42.,  37.,  31.],
         [ 44.,  39.,  33.],
         [ 46.,  41.,  35.]],

        ...,

        [[ 84., 104.,  79.],
         [ 79.,  99.,  74.],
         [ 77.,  99.,  76.],
         ...,
         [ 51.,  76.,  55.],
         [ 54.,  82.,  59.],
         [ 58.,  88.,  62.]],

        [[ 77.,  98.,  79.],
         [ 78., 101.,  81.],
         [ 83., 108.,  89.],
         ...,
         [ 68.,  91.,  75.],
         [ 78., 103.,  84.],
         

In [15]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)

2022-10-17 11:34:28.303482: E tensorflow/stream_executor/cuda/cuda_dnn.cc:389] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2022-10-17 11:34:28.303541: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at conv_ops.cc:1134 : UNIMPLEMENTED: DNN library is not found.


UnimplementedError: Exception encountered when calling layer "conv1" "                 f"(type Conv2D).

{{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:GPU:0}} DNN library is not found. [Op:Conv2D]

Call arguments received by layer "conv1" "                 f"(type Conv2D):
  • inputs=tf.Tensor(shape=(8, 128, 128, 3), dtype=float32)

In [ ]:
base_model.trainable = False

In [ ]:
#base_model.summary()

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [ ]:
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
len(model.trainable_variables)

In [ ]:
initial_epochs = 10
history = model.fit(train_ds,
                    epochs=initial_epochs,
                    validation_data=val_ds)

In [ ]:
loss0, accuracy0 = model.evaluate(val_ds)